In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from IPython.display import display, HTML, Markdown, Image, Video
from ipywidgets import interact, FloatSlider, IntSlider, Button, Output 
display(HTML("<style>.rendered_html.text_cell_render {max-width:700px; }</style>")) 

# Diffusion 101

Target Audience: Mathy and Technical...?

Diffusion models play a major role in the recent success of AI systems 
that can create realistic images from natural language prompts; they lie at the core of
OpenAI's [DALL·E 2](https://openai.com/dall-e-2/) model for instance. 
(Maybe add some more references here...)

In this blog post we review the core ideas behind those models and 
mainly follow the thoughts laid out in 
> [Denoising Diffusion Probabilistic Models](https://arxiv.org/abs/2006.11239), Ho et al., 2020

However, we're bypassing the [ELBO](https://en.wikipedia.org/wiki/Evidence_lower_bound) (Evidence Lower Bound) perspective and take a "short cut" that gets us to the simplified training loss (Eq. 14) in  [[Ho et. al](https://arxiv.org/abs/2006.11239)]. To us the approach presented here feels more intuitive than a detour through ELBO. We might lay out the more technically involved strategy in a different post, but for now we'd like to add a perspective that the reader might not have seen already.

Note that the core idea is *relatively* &mdash; depending on your background of coures &mdash; straight forward, and the *actual* challenge to make a system like DALL·E 2 work at scale seems to be one of engineering really. Nevertheless, we are going to walk you through building a simple diffusion model that creates MNIST digits.

## The Core Idea

Loosely speaking &mdash; and paraphrasing wikipedia here &mdash; *diffusion* is the process of something *spreading out* from a point of higher concentration into its lower concentrated surrounding. 
Usually this process goes along with an increase in entropy (informally the number of microstates in a macro arrangement), or, loosely speaking, a loss of meaning. Reversing the process would enable us to create something concrete out of something meaningless.

A bit more formally, in the context of this post and the above-mentioned [paper](https://arxiv.org/abs/2006.11239), a diffusion process starts with an initial data distribution and defines a Markov chain with Gaussian transition kernels adding noise at each step along the chain. The kernels are chosen such that the further along in the chain we are the more the samples look normally distributed. Reversing this process we can generate new data samples by starting with Gaussian noise and inferring its origin.
<!--     
- ... increases entropy, destroys "meaning"?, what is the right terminology here? because entropy is a measure of information (from Shannon's perspective), avg code length, think that is very different from the physics viewpoint, 
number of micro states in a macro arrangement or something???? 
-->



## The forward process

Ingredients to define the forward process: 
- $Q_0(x)$ - The data (distribution).
- $T$ - The number of steps in the chain.
- $\alpha_1, \ldots, \alpha_T$ - The scale factors for the transition kernels.

Suppose we have a data set $X$ or even better a data distribution $Q_0(x)$ we can sample from.
The (forward) diffusion process is defined as a Markov chain with Gaussian transition kernel as follows: We first sample an initial data point $x_0$ from $Q_0)$, then, we successively add Gaussion noise, i.e. for $t=1,\ldots,T$ we iteratively apply a Gaussian transition kernel and define

$$
    x_t  \ \sim \ Q(x_t \mid x_{t-1}) := N \big( x_t ; \mu_t(x_{t-1}), \Sigma_t \big),
$$

where 

$$
    \mu_t(x_{t-1}) := \sqrt{\alpha_t} \cdot x_{t-1} \ \ \ \ 
            \text{and} \ \ \ \ \Sigma_t :=  (1 - \alpha_t) \cdot I.
$$ 

The factors $\alpha_1, \ldots, \alpha_T$ are chosen such that at the final stage of the chain the $x_T$'s are approximately normal-distributed &mdash; more on that below &mdash; i.e. we want our kernels to satisfy 

$$
    Q(x_T \mid x_0 ) = \int_{x_1,\ldots,x_{T-1}} \prod_{t=1}^T  Q(x_t \mid x_{t-1}) \ \ \text{d}x_0\cdots \text{d}x_{T-1}  \cdot Q_0(x_0)  \stackrel{!}{\approx} \ N(0,I).
$$

Since all the kernels are Gaussian we can compute the distributions at all intermediate stages. With
$\bar\alpha_t := \prod_{t'=1}^t \alpha_{t'}$ one can compute

$$
    Q_t(x_t \mid x_0 ) = \ldots = N \big( x_t ; \sqrt{\bar\alpha}\cdot x_0 , \sqrt{1 - \bar\alpha} \cdot I \big).
    \tag{1}
$$

Thus the above condition translates 
to choosing $\alpha_1,\ldots,\alpha_T$ such that 

$$
    \bar\alpha_T = \prod_{t=1}^T \alpha_{t} \stackrel{!}{\approx} 0 .
$$


## The backwards process

...We're bypassing the ELBO perspective and take a "short cut" to the
simplified training loss in the paper. To us the approach presented here feels more intuitive
than a detour through ELBO. In the end we'd like to add a perspective that 
the reader might not have seen already...

### A short cut to the training objective (without ELBO)

Now that we defined the forward process we can work our way towards reversing it.
Our goal is to model the reverse kernel $Q(x_{t-1} \mid x_t)$. 
Let's choose our approximate model $P_\theta$ to be a diagonal Gaussian and set

$$
        P_\theta(x_{t-1} \mid x_t) = N \big( x_{t-1}; \mu_\theta(x_{t}), \sigma_t \cdot I \big).
$$

We will first focus on the mean $\mu_\theta(x_{t})$ of the model and ignore the parameters $\sigma_1,\ldots, \sigma_T$ for now.

> **Remark: Naive approach is not efficient.**
> The first thing one might try is creating training data by
> repeatedly sampling an initial point $x_0$ and rolling out 
> the Markov chain to get samples $x_0, \ldots, x_T$.
> Then one could simply train the model to map $x_t$ to $x_{t-1}$. 
> 
> While the above training receipe seems straight forward enought this scheme isn't very sample efficient. For 
> $\mu_\theta(x_t)$ to converge to the true mean $\mu^\ast$ we need a quite a 
> lot of sample paths that all pass through $x_t$. To tackle this efficiency problem we are going to 
> identify a better target to update $\mu_\theta$ at each step in the 
> training process.

It is worth recalling what we would love to do: 
Given a sample $x_t$ from $Q(x_t)$, which we get by first sampling $x_0 \sim Q_0$ and then $x_t \sim Q(x_t \mid x_0)$, the desired target signal for the reverse mean model $\mu_\theta(x_t)$ would be given by the mean $\mu^\ast(x_t)$ of the true reverse model $Q(x_{t-1} \mid x_t)$, ie. our objective would be to minimize something along the lines of

$$
    L^\ast(x_t,x_0) := \| \mu^\ast(x_t, x_0) - \mu_\theta(x_t) \|^2.
$$


However we don't have access to $Q(x_{t-1} \mid x_t)$ in closed form. Applying Bayes' rule doesn't help us either since both $Q(x_{t-1})$ and $Q(x_t)$ have no obvious closed form. 
Fortunately we can rewrite $Q(x_{t-1} \mid x_t)$ as

$$
    Q(x_{t-1} \mid x_t) = \int_{x_0} Q(x_{t-1} , x_0 \mid x_t) \ \text{d}x_0.
                        = \int_{x_0} Q(x_{t-1} \mid x_0, x_t)\ Q(x_0 \mid x_t) \ \text{d}x_0.
    \tag{2}
$$

Applying Bayes' rule and the Markov property of the chain
one can show that the first term $Q(x_{t-1} \mid x_0, x_t)$ in the integral can be 
written as the product and quotient of three Gaussians and can in turn 
be expressed as a Gaussian itself. That means there 
is $\tilde\mu(x_t, x_0)$ that we can explicitly write down (see Eq ? in paper) and $\beta_t$ such that we have

$$
    Q(x_{t-1} \mid x_0, x_t) 
                = N\big( x_t ; \tilde\mu(x_t, x_0), \tilde\beta_t\cdot I \big).
    \tag{3}
$$

With this in hand we can use Eq. 2 to express the true target $\mu^\ast(x_t)$
as an expected value of assessible intermediate targets $\tilde\mu(x_t, x_0)$
(cf. Figure 1 below); we have

$$
    \mu^\ast(x_t) = \int_{x_0}  \tilde\mu(x_t, x_0) \ Q(x_0 \mid x_t) \ \text{d}x_0.
    \tag{4}
$$

> <img src="Diffusion_target.png" width="500">
>
> **Figure 1:** True traget in magenta. Approximate target as a sum of partial targets in blue, 
> weighted by their probability of occurence in orange. The black lines indicate sample paths 
> through $x_t$ starting at $x_0$ and $x_0'$ respectively.

Note, we can use $\tilde\mu(x_t, x_0)$ as a crude approximation for $\mu^\ast(x_t)$. Practically this is quite convenient, since we needed to sample $x_0 \sim Q_0$ first to be able to get a sample $x_t \sim Q(x_t \mid x_0)$ anyway.

> **Remark: Why crude approximation.**
> Technically, we sampled $x_0,x_t \sim Q(x_t, x_0) = Q(x_t \mid x_0)Q(x_0)$. 
> Now suppose we got a trillion of those samples and suppose further $x_t$ takes 
> only finiteley many discrete values so we could bin the 
> pairs with respect to a fixed $x_t$. Taking all pairs 
> $(x^1_0,x_t), \ldots, (x^n_0,x_t)$ for a fixed $x_t$ we can 
> approximate the integral by $\tfrac{1}{n}\sum_i \tilde\mu(x_t, x^i_0)$. 
> In practice it is very (very) unlikely to hit some $x_t$ twice, 
> so we find ourselves stuck with $n=1$ in the approximation.


From this viewpoint it makes somewhat sense that our objective becomes
minimizing the distance between $\tilde\mu$ and $\mu_\theta$, or equivalently 
minimizing

$$
    \tilde{L}(x_t,x_0) := \| \tilde\mu(x_t, x_0) - \mu_\theta(x_t) \|^2.
    \tag{4}
$$





### Simplifying the model to predict noise

All that is left is to end up with the simplified loss in [[Ho et. al](https://arxiv.org/abs/2006.11239)]
is to rephrase the objective in terms of $x_0$, $t$, and $\epsilon$. Recall form Eq. 1 that $Q(x_t \mid x_0)$ is a diagonal Gaussian and we can generate samples by sampling $\epsilon \sim N(0,I)$ first and setting

$$
    x_t = x_t(x_0, t, \epsilon) = \sqrt{\bar\alpha_t}\cdot x_0 + \sqrt{1- \bar\alpha_t}\cdot \epsilon.
    \tag{5}
$$

Moreover, suppose we knew $\epsilon$, $t$, and $x_t$ then we can solve for $x_0$ and get

$$
    x_0 = x_0(x_t, \epsilon, t) = 
        \frac{1}{ \sqrt{\bar\alpha_t}}\big( x_t - \sqrt{1- \bar\alpha_t}\cdot \epsilon \big).
$$

This enables us to do two things: First, it means we can express our training target $\tilde\mu(x_t, x_0)$ as a function of $x_0$, $t$, and $\epsilon$. Our training target becomes 

$$
    \tilde\mu(x_t, x_0) = \tilde\mu(x_0, t, \epsilon).
$$

Second, to model $\tilde\mu$ we really only need to model the $\varepsilon$ term.
That means with such a model $\epsilon_\theta(x_t,t)$, we could simply
define 

$$
    \mu_\theta(x_t, t) := \tilde\mu(x_0, t, \epsilon_\theta(x_t, t)).
$$

For this to be a good model we need to fit $\epsilon_\theta$ and minimize our final objective
(cf. Eq. 14 in [[Ho et. al](https://arxiv.org/abs/2006.11239)])

$$
    L_{\text{simple}} =
        \mathbb{E}_{x_0,t,\epsilon} \Big[ \big\| 
            \epsilon - \epsilon_\theta\big( x_t(x_0, t, \epsilon),t \big) 
        \big\|^2   \Big].
$$